In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm

In [17]:
from selenium import webdriver

In [18]:
def musinsa_rank(category_num,page_num):
    url =  f"https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=00{category_num}&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page={page_num}&viewType=small&priceMin=&priceMax="
    response = requests.get(url)
    html = bs(response.text, 'lxml')
    musinsa_rank_df = rbnl(html)

    return musinsa_rank_df

In [19]:
def rbnl(html):
    musinsa_rank_df = pd.DataFrame()

    #순위 뽑기
    rank_html = html.select('#goodsRankList > li > p')
    rank_no_list = []

    for i in rank_html:
        rank_no_list.append(i.string.strip())

    musinsa_rank_df['순위'] = rank_no_list

    #브랜드 이름 뽑기
    brand_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.item_title > a')
    brand_list = []

    for i in brand_html:
        brand_list.append(i.string)



    #링크와 의류명 뽑기
    link_name_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.list_info > a')
    link_list = []
    name_list = []

    for i in link_name_html:
        link_list.append(i['href'])
        name_list.append(i['title'])

    musinsa_rank_df['상품명']=name_list
    musinsa_rank_df['브랜드명']=brand_list
    musinsa_rank_df['url']=link_list

    #상세 페이지 크롤링
    musinsa_rank_df2 = specific_info(link_list)

    #데이터 프레임 옆으로 합치기
    musinsa_rank_df = pd.concat([musinsa_rank_df, musinsa_rank_df2], axis=1)

    return musinsa_rank_df

In [20]:
from pyasn1.compat.octets import null


def specific_info(link_list):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}

    musinsa_rank_df = pd.DataFrame()
    part_num_list=[]
    sex_list=[]
    view_list=[]
    sales_list=[]
    like_list=[]
    review_list=[]
    star_list=[]


    for link in tqdm(link_list):
        #         print(link)
        response_1=requests.get(link,headers=headers)
        html_1=bs(response_1.text, 'lxml')

        #품번 리스트 생성
        part_num_html=html_1.select('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong')
        part_num=part_num_html[0].get_text().split('/')[-1].strip()
        part_num_list.append(part_num)

        #성별 리스트 생성
        sex_html=html_1.select("#product_order_info > div.explan_product.product_info_section > ul > li > p.product_article_contents > span.txt_gender")
        sex=sex_html[0].get_text().replace('\n',' ').strip()
        sex_list.append(sex)

        #셀레니움으로 원하는 데이터 가져오기
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(link)
        sel_html=driver.page_source
        html_2=bs(sel_html)

        #조회수 가져오기
        view_html=html_2.find_all("strong", {"id":"pageview_1m"})
        view=view_html[0].get_text()
        view_list.append(view)

        #누적 판매 가져오기
        sales_html=html_2.find_all("strong", {"id":"sales_1y_qty"})
        sales=sales_html[0].get_text()
        sales_list.append(sales)

        #좋아요 수 가져오기
        like_html=html_2.find_all("span", {"class": "prd_like_cnt"})
        like=like_html[0].get_text()
        like_list.append(like)

        #평점
        star_html=html_2.find_all("span", {"class": "prd-score__rating" })
        if len(star_html) == 0:
            star = "평점 0점"
        else:
            star=star_html[0].get_text()
        star_list.append(star)


        #구매 후기 개수 가져오기
        review_html=html_2.find_all("span", {"class": "prd-score__review-count"})
        if len(review_html) == 0:
            review = "후기 0개"
        else:
            review=review_html[0].get_text()
        review_list.append(review)


        #드라이버 닫아주기
        driver.close()

        #시간 추가
        time.sleep(0.01)


    musinsa_rank_df['조회수']=view_list
    musinsa_rank_df['좋아요']=like_list
    musinsa_rank_df['구매후기 개수']=review_list
    musinsa_rank_df['평점']=star_list
    musinsa_rank_df['누적판매량(1년)']=sales_list


    return musinsa_rank_df

In [26]:
category_num=input("Category 번호를 입력하세요: ")
page_num=input("Page 번호를 입력하세요: ")
final_df=musinsa_rank(category_num,page_num)

  0%|          | 0/90 [00:00<?, ?it/s]C:\Users\jpc10\AppData\Local\Temp\ipykernel_19664\3077604374.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
100%|██████████| 90/90 [15:07<00:00, 10.08s/it]


In [27]:
file_name = f"0319 musinsa_ranking_category{category_num}_page{page_num}.csv"
final_df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,순위,상품명,브랜드명,url,조회수,좋아요,구매후기 개수,평점,누적판매량(1년)
0,271위,CLASSIC LOGO HOODIE light heather gray,마리떼,https://www.musinsa.com/app/goods/2328168?loc=...,1.4천 회 이상,665,후기 64개 보기,4.9,100개 이상
1,272위,Pure supima boat neck pullover_Noon,우마뭉,https://www.musinsa.com/app/goods/3081973?loc=...,NaN,14,후기 0개,평점 0점,NaN
2,273위,NEW CLASSIC LOGO HOODIE navy,마리떼,https://www.musinsa.com/app/goods/3035965?loc=...,1.3천 회 이상,138,후기 7개 보기,5,50개 미만
3,274위,[SET] (맨투맨 OR 후드 선택) CGP 아치 로고 트레이닝 셋업_그레이,코드그라피,https://www.musinsa.com/app/goods/2096583?loc=...,2.6만 회 이상,"21,713","후기 1,444개 보기",4.8,4.2천 개 이상
4,275위,[2PACK] 코튼 크루넥 반팔_SPRWCA9U01,스파오,https://www.musinsa.com/app/goods/2628879?loc=...,5.6천 회 이상,"2,359",후기 641개 보기,4.8,3천 개 이상
...,...,...,...,...,...,...,...,...,...
85,356위,벨라 스웻셔츠_NAVY,그로브,https://www.musinsa.com/app/goods/3138601?loc=...,2.2천 회 이상,197,후기 0개,평점 0점,50개 미만
86,357위,Shield Artwork Sweatshirt in Blue VW2AE335-22,레이브,https://www.musinsa.com/app/goods/2826164?loc=...,600회 이상,368,후기 15개 보기,4.5,50개 미만
87,358위,스몰 트위치 로고 에어로웜 오버핏 후드 티셔츠 멜란지 그레이,리,https://www.musinsa.com/app/goods/1571510?loc=...,4.8천 회 이상,"33,598","후기 3,663개 보기",4.8,4.1천 개 이상
88,359위,탑필 울 라운드 니트 (블랙),쿠어,https://www.musinsa.com/app/goods/1548435?loc=...,3.2만 회 이상,"16,115","후기 6,932개 보기",4.9,8.6천 개 이상
